In [ ]:
import numpy as np
import xmltodict
import json
import os
import pickle

In [ ]:
num = 0
count = 0
inside = []
name = "./data/output_event_"
with open("./output_events.xml", 'rt') as f:
    a = f.readline()
    b = f.readline()
    for line in f.readlines():
        inside.append(line)
        num += 1
        if num % 5000000 == 0:
            count += 1
            xml = []
            xml.append(a)
            xml.append(b)
            xml.extend(inside)
            if inside[-1] != "</events>":
                xml.append("</events>")
            xml_file = ''.join(xml)
            js = xmltodict.parse(xml_file, encoding='utf-8')
            file_name = name + str(count) + "_" + str(num) + ".json"
            with open(file_name, 'w') as fw:
                json.dump(js, fw)
            inside = []
            del js, xml_file


In [ ]:
# deal with the rest
count += 1
xml = []
xml.append(a)
xml.append(b)
xml.extend(inside)
if inside[-1] != "</events>":
    xml.append("</events>")
xml_file = ''.join(xml)
js = xmltodict.parse(xml_file, encoding='utf-8')
file_name = name + str(count) + "_" + str(num) + ".json"
with open(file_name, 'w') as fw:
    json.dump(js, fw)

In [ ]:
events = os.listdir("./data")
events.remove(".DS_Store")
ordered_events = [0] * len(events)

In [ ]:
for event in events:
    idx = int(event.split("_")[2]) - 1
    ordered_events[idx] = event

In [ ]:
eventList = []
for event in ordered_events:
    path = "./data/" + event
    with open(path, 'r') as f:
        json_event = json.load(f)
    eventList.extend(json_event["events"]["event"])

In [ ]:
# pre-process data(remove all the public transportation dataset)
# event is recorded in this form {{"time1":time1, "personId":person, "linkId":"link"}, ...}
eventsWithoutPublic = []
for item in eventList:
    if item.__contains__("@link"):
        if item.__contains__("@person"): 
            if not item["@person"].startswith("pt"):
                eventsWithoutPublic.append({"time":float(item["@time"]), "person":item["@person"], "link":item["@link"]})
        else:
            if item.__contains__("@vehicle"):
                if not item["@vehicle"].startswith("pt"):
                    eventsWithoutPublic.append({"time":float(item["@time"]), "person":item["@vehicle"], "link":item["@link"]})
    

In [ ]:
with open("event_without_public.json", 'w') as ewp:
    json.dump(eventsWithoutPublic, ewp)